# 2. Tensorflow

In [1]:
import os
import tempfile
import tensorflow as tf

from tensorflow.keras.datasets import imdb #imdb 데이터셋을 불러온다
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import utils

import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)

/Users/sinseongjin/tf110/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


1.10.1


## 2-1 Data Prep

In [2]:
VOCAB_SIZE = 10000 #문장의 단어 사이즈
SENT_SIZE = 200 #문장 길이
BATCH_SIZE = 128

PAD_ID = 0 # 200단어 이하 문장에 대해서 0 값을 채워 넣는다.
START_ID = 1 # 시작 id 값
OOV_ID = 2 # out of vocab
INDEX_OFFSET = 2


#IMDB 데이터셋을 로드 합니다. 학습과 테스트 셋으로 나눕니다.
(xTrainVar, yTrain), (xTestVar, yTest) = imdb.load_data(num_words=VOCAB_SIZE,
                                                      start_char=START_ID,
                                                      oov_char=OOV_ID,
                                                      index_from=INDEX_OFFSET)

# 패딩 채워주기
print("Pad sequences (samples x time)")
xTrain = sequence.pad_sequences(xTrainVar, 
                                 maxlen=SENT_SIZE,
                                 truncating='post',
                                 padding='post',
                                 value=PAD_ID)

xTest = sequence.pad_sequences(xTestVar, 
                                maxlen=SENT_SIZE,
                                truncating='post',
                                padding='post', 
                                value=PAD_ID)

xLenTrain = np.array([min(len(x), SENT_SIZE) for x in xTrainVar])
xLenTest = np.array([min(len(x), SENT_SIZE) for x in xTest])

print("xTrain shape:", xTrain.shape)
print("xTest shape:", xTest.shape)

# Pad sequences (samples x time)
# xTrain shape: (25000, 200)
# xTest shape: (25000, 200)

Pad sequences (samples x time)
xTrain shape: (25000, 200)
xTest shape: (25000, 200)


In [7]:
VOCAB_SIZE = 10000 #문장의 단어 사이즈
SENT_SIZE = 200 #문장 길이
EMB_SIZE = 128 #임베딩 사이즈
MODEL_DIR = tempfile.mkdtemp() #임시 디렉토리에 모델 저장

PAD_ID = 0 # 200단어 이하 문장에 대해서 0 값을 채워 넣는다.
START_ID = 1 # 시작 id 값
OOV_ID = 2 # out of vocab
INDEX_OFFSET = 2

#IMDB 데이터셋을 로드 합니다. 학습과 테스트 셋으로 나눕니다.
(xTrainVar, yTrain), (xTestVar, yTest) = imdb.load_data(num_words=VOCAB_SIZE,
                                                      start_char=START_ID,
                                                      oov_char=OOV_ID,
                                                      index_from=INDEX_OFFSET)

# 패딩 채워주기
print("Pad sequences (samples x time)")
xTrain = sequence.pad_sequences(xTrainVar, 
                                 maxlen=SENT_SIZE,
                                 truncating='post',
                                 padding='post',
                                 value=PAD_ID)

xTest = sequence.pad_sequences(xTestVar, 
                                maxlen=SENT_SIZE,
                                truncating='post',
                                padding='post', 
                                value=PAD_ID)

print("xTrain shape:", xTrain.shape)
print("xTest shape:", xTest.shape)

xLenTrain = np.array([min(len(x), SENT_SIZE) for x in xTrainVar])
xLenTest = np.array([min(len(x), SENT_SIZE) for x in xTest])

Pad sequences (samples x time)
xTrain shape: (25000, 200)
xTest shape: (25000, 200)


## 2.2 tf.data

  머신러닝에서 많은 시간을 소요하는 것은 데이터를 다루는 것이다. 일반적으로 데이터 분석, 전처리, 파이프라인 만드는 과정에서 약 70~80%의 시간을 소비한다 해도 과언이 아니다. 텐서플로우에서는 Dataset API를 활용하여, 단순한 연구자 뿐만 아니라 서비스화를 위해서 데이터에 대한 고민들을 해결 해 주고 있다. 학습 속도를 올리기 위해  GPU가 병목현상 없이 효율적으로 최적화를 지원하는 것 뿐만 아니라 이미지, 텍스트를 넘어 넘파이 (Numpy) 및 판다스(Pandas)데이터 구조가 활용하도록 지원이 가능하다.
  
  tf.data를 사용하기 위한 데이터를 활용하기 위해, IMDB 영화 리뷰 데이터셋을 불러오자. 데이터셋은 2만5천개의 리뷰의 데이터가 긍/부정으로 나누어져 있고, 전처리도 되어 있다. IMDB 영화 리뷰 데이터셋을 선정한 이유는 MNIST와 처럼 데이터셋을 tensorflow-keras를 활용하여 가장 손쉽게 불러오는 자연어 데이터이기 때문이다.

In [3]:
# IMDB데이터셋을 학습 데이터셋으로 로드 한다.

# Dataset에서 Iterator를 생성하여 초기화를 하거나 그 다음에 있는 tf.Tensor 객체를 생성하자.
#학습 셋을 Dataset에 입력한다. 입력 형태는 numpy 형태이다.
dataset = tf.data.Dataset.from_tensor_slices((xTrain, xLenTrain, yTrain))
iterator = dataset.make_one_shot_iterator()
nextData = iterator.get_next()

# 세션을 실행 시켜서 간단한 테스트를 진행 해 보자
with tf.Session() as sess:
    sentIdx, sentLen, sentLabel = sess.run(nextData)
    print("sentence index: {}".format(sentIdx)) # 입력 문장
    print("sentence lenght: {}".format(sentLen)) # 입력 문장의 길이
    print("sentence Label: {}".format(sentLabel)) # 입력 문장 레이블 (긍/부정))
    
# 기본적인 기능 이외에도 shuffle, repeat, batch를 설정하여 데이터의 구조를 다양하게 변경 가능하다
dataset = dataset.shuffle(buffer_size=len(xTrain)) # 데이터를 섞어준다.
dataset = dataset.repeat() # 데이터를 반복 진행한다
dataset = dataset.batch(BATCH_SIZE) # 데이터를 배치 형태로 생성한다

sentence index: [   1   13   21   15   42  529  972 1621 1384   64  457 4467   65 3940
    3  172   35  255    4   24   99   42  837  111   49  669    2    8
   34  479  283    4  149    3  171  111  166    2  335  384   38    3
  171 4535 1110   16  545   37   12  446    3  191   49   15    5  146
 2024   18   13   21    3 1919 4612  468    3   21   70   86   11   15
   42  529   37   75   14   12 1246    3   21   16  514   16   11   15
  625   17    2    4   61  385   11    7  315    7  105    4    3 2222
 5243   15  479   65 3784   32    3  129   11   15   37  618    4   24
  123   50   35  134   47   24 1414   32    5   21   11  214   27   76
   51    4   13  406   15   81    2    7    3  106  116 5951   14  255
    3    2    6 3765    4  722   35   70   42  529  475   25  399  316
   45    6    3    2 1028   12  103   87    3  380   14  296   97   31
 2070   55   25  140    5  193 7485   17    3  225   21   20  133  475
   25  479    4  143   29 5534   17   50   35   27  223   91 

In [9]:
# 기본적인 관련 코드에 대한 설명 구조

def parser(input_sent, length, label):
    features = {"input_sent": input_sent, "len": length} #문장의 입려과 길이를 딕셔너리 형태로 만들었다.
    return features, label

def train_input_fn():
    #학습을 위한 tf.dataset 함수 구조를 선언한다.
    with tf.device('/cpu:0'): #GPU학습과 CPU학습 구조를 분리하기 위해서 데이터 전처리는 CPU에서 가능하도록 지정한다.
        dataset = tf.data.Dataset.from_tensor_slices((xTrain, xLenTrain, yTrain))
        dataset = dataset.shuffle(buffer_size=len(xTrain)) #데이터를 무작위로 섞는 기능을 한다. buffer_size는 데이터가 아주 클 때 활용한다.
        dataset = dataset.repeat() # 반복기능을 통해 학습 Epoch 까지 반복한다.
        dataset = dataset.map(parser) #향후 Estimator 등의 기능에서 조금 더 쉽게 데이터를 선택하기 위해 dict 구조로 만들어준다.
        dataset = dataset.batch(BATCH_SIZE) # 배치 사이즈를 설정한다.
        iterator = dataset.make_one_shot_iterator() # 위에서 한번 언급하였던 Simple하게 데이터를 불러오는 법이다.
    return iterator.get_next() #get_next()를 통하여 데이터값을 불러온다.

with tf.Session() as sess:
    print(sess.run(train_input_fn())) #간단하게 출력해본다.

def eval_input_fn():
    with tf.device('/cpu:0'):
        dataset = tf.data.Dataset.from_tensor_slices((xTest, xLenTest, yTest))
        dataset = dataset.map(parser)
        dataset = dataset.batch(BATCH_SIZE)
        iterator = dataset.make_one_shot_iterator()
        return iterator.get_next()

({'input_sent': array([[   1,   12,  218, ...,    0,    0,    0],
       [   1,   13,   19, ...,    0,    0,    0],
       [   1, 1360,   17, ...,    7,  168,   45],
       ...,
       [   1, 7337,  810, ...,  200,   28,    2],
       [   1,   13,   21, ...,    0,    0,    0],
       [   1,   12,  123, ...,   70,  415,   22]], dtype=int32), 'len': array([107,  55, 200, 138,  61, 200,  85, 117, 200, 132, 200, 152, 200,
       200, 104, 140, 200, 139, 200, 200, 200, 200, 107, 186, 200, 199,
       200, 108, 200, 200,  47, 143,  45, 132,  72,  77, 200, 106, 200,
       157, 200, 200, 200, 128,  69, 200, 137, 130, 149, 200, 148, 200,
       140, 200, 141, 200, 121, 200, 200, 127, 186, 200, 182, 177, 177,
       200, 169, 200, 183,  69,  37, 177, 174, 145,  40, 154, 200, 198,
       152,  52,  64, 132, 159, 200, 200, 200, 200, 200, 200, 149, 128,
       200, 200,  58, 157, 168, 200, 177,  98, 200, 151, 200, 200, 131,
       158, 135, 200, 200, 135, 179, 200,  81, 187, 200, 200, 119, 200,
  

In [ ]:
#!/usr/bin/env python3
#-*- coding: utf-8 -*-
import os
import tempfile
import tensorflow as tf
import sys


from tensorflow.python.keras.datasets import imdb #imdb dataset
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import utils
from tensorboard import summary as summary_lib
import numpy as np

tf.logging.set_verbosity(tf.logging.INFO)
print(tf.__version__)
os.environ["CUDA_VISIBLE_DEVICES"]="0" #For TEST

vocab_size = 5000
sentence_size = 200
embedding_size = 50
model_dir = tempfile.mkdtemp()

NUM_EPOCHS = 10
BATCH_SIZE = 128
HIDDEN_DIM = 32
KEEP_PROB = 0.3

pad_id = 0
start_id = 1
oov_id = 2
index_offset = 2
​
(x_train_variable, y_train), (x_test_variable, y_test) = imdb.load_data(num_words=vocab_size,
                           start_char=start_id,
                           oov_char=oov_id,
                           index_from=index_offset)
​
print("x_train shape", x_train_variable.shape)
print("x_test shape", x_test_variable.shape)
​
print("Pad sequences (samples x time)")
​
x_train = sequence.pad_sequences(x_train_variable, 
                 maxlen=sentence_size,
                 truncating='post',
                 padding='post',
                 value=pad_id)
x_test = sequence.pad_sequences(x_test_variable, 
                maxlen=sentence_size,
                truncating='post',
                padding='post', 
                value=pad_id)
print("x_train shape:", x_train.shape)
print("x_test shape:", x_test.shape)
​
x_len_train = np.array([min(len(x), sentence_size) for x in x_train_variable])
x_len_test = np.array([min(len(x), sentence_size) for x in x_test_variable])
​
word_index = imdb.get_word_index()
word_inverted_index = {v + index_offset: k for k, v in word_index.items()}
​
# The first indexes in the map are reserved to represent things other than tokens
word_inverted_index[pad_id] = '<PAD>'
word_inverted_index[start_id] = '<START>'
word_inverted_index[oov_id] = '<OOV>'
​
# for i in range(0, 10):
#   print(i, word_inverted_index[i])
  
def index_to_text(indexes):
  return ' '.join([word_inverted_index[i] for i in indexes])
​
# print(index_to_text(x_train_variable[0]))
​
​
def parser(input_sent, length, label):
  features = {"input_sent": input_sent, "len": length}
  return features, label
​
#the former (repeat before shuffle) provides better performance, 
# while the latter (shuffle before repeat) provides stronger ordering guarantees.
​
def train_input_fn():
  with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.shuffle(buffer_size=len(x_train))
    dataset = dataset.repeat()
    dataset = dataset.map(parser)
    dataset = dataset.batch(BATCH_SIZE)
​
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()
​
def eval_input_fn():
  with tf.device('/cpu:0'):
    dataset = tf.data.Dataset.from_tensor_slices((x_train, x_len_train, y_train))
    dataset = dataset.map(parser)
    dataset = dataset.batch(BATCH_SIZE)
    iterator = dataset.make_one_shot_iterator()
    return iterator.get_next()
​
def gru_classifier(features, labels, mode, params):
  
  TRAIN = mode == tf.estimator.ModeKeys.TRAIN
  EVAL = mode == tf.estimator.ModeKeys.EVAL
  PREDICT = mode == tf.estimator.ModeKeys.PREDICT
  
  input_layer = tf.contrib.layers.embed_sequence(
                        features['input_sent'],
                        vocab_size,
                        embedding_size,
                        initializer=params['embed_init']
                        )
  
  with tf.variable_scope('bi-directional_gru'):
    gru_fw_cell = tf.contrib.rnn.GRUCell(num_units = HIDDEN_DIM,
                           activation = tf.nn.tanh)
    gru_bw_cell = tf.contrib.rnn.GRUCell(num_units = HIDDEN_DIM,
                           activation = tf.nn.tanh)
    gru_fw_cell = tf.contrib.rnn.DropoutWrapper(gru_fw_cell, KEEP_PROB)
    gru_bw_cell = tf.contrib.rnn.DropoutWrapper(gru_bw_cell, KEEP_PROB)
​
    _, output_states = tf.nn.bidirectional_dynamic_rnn(cell_fw = gru_fw_cell, cell_bw = gru_bw_cell,
                            inputs = input_layer, sequence_length = features['len'],
                            dtype = tf.float32)
    
    concat_gru_output = tf.concat([output_states[0], output_states[1]], axis = 1)
​
  with tf.variable_scope('output_layer'):
    score = tf.layers.dense(concat_gru_output, 1, name='score_layer')
      
  if labels is not None:
    # labels = tf.one_hot(labels,2)
    labels = tf.reshape(labels, [-1, 1])
  # print(labels)
  
  optimizer = tf.train.AdamOptimizer()
  
  if TRAIN:
    global_step = tf.train.get_global_step()
    loss = tf.losses.sigmoid_cross_entropy(labels, score)
    train_op = tf.train.AdamOptimizer(1e-4).minimize(loss, global_step)
    return tf.estimator.EstimatorSpec(mode=mode, train_op=train_op, loss = loss)
  
  elif EVAL:
    loss = tf.losses.sparse_softmax_cross_entropy(labels, score)
    pred = tf.nn.sigmoid(score)
    accuracy = tf.metrics.accuracy(labels, tf.round(pred))
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops={'acc': accuracy})
    
  elif PREDICT:
    return tf.estimator.EstimatorSpec(
      mode=mode,
      predictions={
        'prob': score,
      }
    )
​
​
params = {'embed_init': tf.random_uniform_initializer(-1.0, 1.0)}
​
# Create Checkpoint Folder
model_dir = os.path.join(os.getcwd(), "data_out/gru")
os.makedirs(model_dir, exist_ok=True)
​
# Set Config
config_tf = tf.estimator.RunConfig()
# config_tf._save_checkpoints_steps = 100
config_tf._keep_checkpoint_max = 2
# config_tf._log_step_count_steps = 100
​
gru_classifier = tf.estimator.Estimator(model_fn = gru_classifier,
                    model_dir='./data_out/gru',
                    params=params,
                    config=config_tf)
​
def train_and_evaluate(classifier):
  classifier.train(input_fn=train_input_fn, steps=NUM_EPOCHS)
  eval_result = classifier.evaluate(input_fn=eval_input_fn)
​
train_and_evaluate(gru_classifier)
​
# 예측된 모델을 불러 체크포인트로 결과치를 불러온다.
​
def get_predictions(estimator, input_fn):
  return [x["prob"][0] for x in estimator.predict(input_fn=input_fn)]
​
LABELS = ["negative", "positive"]
​
# predict_input_fn = tf.estimator.inputs.numpy_input_fn(x={"input_sent":train_input_fn}, shuffle=False)
# gru_classifier.predict(input_fn=predict_input_fn)
​
with tf.Graph().as_default():
  cm = tf.confusion_matrix(y_train, get_predictions(gru_classifier, train_input_fn))
​
  with tf.Session() as session:
    cm_out = session.run(cm)
​
cm_out = cm_out.astype(float) / cm_out.sum(axis=1)[:, np.newaxis]
​
sns.heatmap(cm_out, annot=True, xticklabels=LABELS, yticklabels=LABELS)
plt.xlabel("Predicted")
plt.ylabel("True")
​
# predictions = np.array([p['logits'][0] for p in cnn_classifier.predict(input_fn=predict_input_fn)])
​
# #   # Reset the graph to be able to reuse name scopes
# #   tf.reset_default_graph() 
# #   # Add a PR summary in addition to the summaries that the classifier writes
# #   pr = summary_lib.pr_curve('precision_recall', predictions=predictions, labels=y_test.astype(bool), num_thresholds=21)
# #   with tf.Session() as sess:
# #     writer = tf.summary.FileWriter(os.path.join(classifier.model_dir, 'eval'), sess.graph)
# #     writer.add_summary(sess.run(pr), global_step=0)
# #     writer.close()
​
Collapse 